##検証

###運行情報、天気情報取得に使用するコード

下記のコードをdatabase.py内にて実行し、1時間毎に運行情報と天気情報を取得し、データベース内に保存する

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3
import time
import schedule
import pytz  # タイムゾーンを扱うためのライブラリ

# SQLite DBファイル名
DB_NAME = "最終課題/transport_weather.db"

# テーブル作成
def create_table():
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS transport_weather (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                fetch_time TEXT,
                weather TEXT,
                delay_status TEXT
            )
        ''')
        conn.commit()

# データをDBに追加する関数
def insert_data(date, fetch_time, weather, delay_status):
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO transport_weather (date, fetch_time, weather, delay_status)
            VALUES (?, ?, ?, ?)
        ''', (date, fetch_time, weather, delay_status))
        conn.commit()

# データ取得: 多摩都市モノレールの運行状況
def get_monorail_status():
    TamaMono_URL = 'https://transit.yahoo.co.jp/diainfo/156/0'
    try:
        TamaMono_Requests = requests.get(TamaMono_URL)
        TamaMono_Soup = BeautifulSoup(TamaMono_Requests.text, 'html.parser')
        if TamaMono_Soup.find('dd', class_='trouble'):
            return '多摩都市モノレールは遅延しています'
        else:
            return '多摩都市モノレールは通常運転です'
    except Exception as e:
        return f"運行状況取得エラー: {e}"

# データ取得: 現在の天気情報
def get_current_weather():
    # tenki.jp の天気情報ページ URL
    forecast_url = "https://tenki.jp/forecast/3/16/4410/13212/"
    try:
        response = requests.get(forecast_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        # 現在の天気情報を取得
        current_weather = soup.find('p', class_='weather-telop')
        if not current_weather:
            return "現在の天気情報が見つかりませんでした"
        return current_weather.get_text(strip=True)
    except Exception as e:
        return f"天気情報の取得に失敗しました: {e}"

# データを取得してDBに追加
def fetch_and_store_data():
    # 現在時刻と日付を取得（日本時間）
    now = datetime.now(pytz.timezone('Asia/Tokyo'))
    fetch_time = now.strftime("%H:%M:%S")
    today_date = now.strftime("%Y-%m-%d")
    # 天気と運行状況を取得
    weather = get_current_weather()
    delay_status = get_monorail_status()

    # データをDBに挿入
    insert_data(today_date, fetch_time, weather, delay_status)
    print("データをDBに保存しました:")
    print(f"日付: {today_date}, 時刻: {fetch_time}, 天気: {weather}, 遅延: {delay_status}")

# DB内のデータを表示する関数
def display_db_data():
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM transport_weather")
        rows = cursor.fetchall()
        print("DB内のデータ:")
        for row in rows:
            print(row)

# 定期実行タスクのスケジュール設定
def schedule_tasks():
    # 5:00～23:00の間、1時間ごとに実行
    for hour in range(5, 24):
        time_str = f"{hour:02d}:00"
        schedule.every().day.at(time_str).do(fetch_and_store_data)
    # 翌日00:00のスケジュールを追加
    schedule.every().day.at("00:00").do(fetch_and_store_data)

    print("スケジュールを設定しました: 5:00～24:00の間、1時間ごとに実行します。")

# メイン処理
if __name__ == "__main__":
    create_table()
    schedule_tasks()

    print("スケジュール実行を開始します...")
    while True:
        schedule.run_pending()
        time.sleep(1)

###データベース全体の表示

下記コードを用いてtransport_weather.db内に保存されたデータを表示

In [1]:
import sqlite3
import pandas as pd

# データベースに接続
db_path = "transport_weather.db"
conn = sqlite3.connect(db_path)

# クエリを実行してテーブルを取得
query = "SELECT * FROM transport_weather;"
df = pd.read_sql_query(query, conn)

# Jupyter Notebookで綺麗に表示
from IPython.display import display
display(df)

# データベース接続を閉じる
conn.close()

,id,date,fetch_time,weather,delay_status
0,1,2025-01-19,05:00:00,曇時々晴,多摩都市モノレールは通常運転です
1,2,2025-01-19,06:01:35,曇時々晴,多摩都市モノレールは通常運転です
2,3,2025-01-19,07:05:07,曇時々晴,多摩都市モノレールは通常運転です
3,4,2025-01-19,08:00:00,曇時々晴,多摩都市モノレールは通常運転です
4,5,2025-01-19,09:00:00,曇時々晴,多摩都市モノレールは通常運転です
...,...,...,...,...,...
95,96,2025-01-23,20:00:00,晴,多摩都市モノレールは通常運転です
96,97,2025-01-23,21:12:55,晴,多摩都市モノレールは通常運転です
97,98,2025-01-23,22:00:00,晴,多摩都市モノレールは通常運転です
98,99,2025-01-23,23:00:00,晴,多摩都市モノレールは通常運転です


###1.delay_statusが「多摩都市モノレールは遅延しています」のみ表示

delay_status列において、「多摩都市モノレールは遅延しています」というデータがある列のみ表示

In [ ]:
#データベース接続
db_path = "transport_weather.db"
conn = sqlite3.connect(db_path)

#クエリの実行
query = """
SELECT date, fetch_time, weather, delay_status
FROM transport_weather
WHERE delay_status = '多摩都市モノレールは遅延しています';
"""
df = pd.read_sql_query(query, conn)

#データの有無をチェック
if df.empty:
    print("遅延データはありません")
else:
    print("遅延データを表示します:")
    #Jupyter Notebookで綺麗に表示
    from IPython.display import display
    display(df)

#データベース接続を閉じる
conn.close()

遅延データはありません


###2.weatherに「雨」が含まれている列のみ表示

weather列において、「雨」というデータがある列のみ表示

In [11]:
import sqlite3
import pandas as pd
from IPython.display import display

# データベース接続
db_path = "transport_weather.db"
conn = sqlite3.connect(db_path)

# クエリの実行: weather列に「雨」が含まれているデータを抽出
query = """
SELECT date, fetch_time, weather, delay_status
FROM transport_weather
WHERE weather LIKE '%雨%';
"""
df = pd.read_sql_query(query, conn)

# データの有無をチェック
if df.empty:
    print("「雨」に関連するデータはありません。")
else:
    print("「雨」に関連するデータを表示します:")
    display(df)

# データベース接続を閉じる
conn.close()


「雨」に関連するデータを表示します:


,date,fetch_time,weather,delay_status
0,2025-01-19,12:07:42,曇一時雨,多摩都市モノレールは通常運転です
1,2025-01-19,13:00:00,曇一時雨,多摩都市モノレールは通常運転です
2,2025-01-19,14:08:22,曇一時雨,多摩都市モノレールは通常運転です
3,2025-01-19,15:00:00,曇のち雨,多摩都市モノレールは通常運転です
4,2025-01-19,16:00:00,曇一時雨,多摩都市モノレールは通常運転です
5,2025-01-19,17:00:00,曇一時雨,多摩都市モノレールは通常運転です
6,2025-01-19,18:09:54,曇のち雨,多摩都市モノレールは通常運転です
7,2025-01-19,19:00:00,曇のち雨,多摩都市モノレールは通常運転です
8,2025-01-19,20:09:19,曇のち雨,多摩都市モノレールは通常運転です
9,2025-01-19,21:00:00,曇のち雨,多摩都市モノレールは通常運転です


###検証

「1.delay_statusが「多摩都市モノレールは遅延しています」のみ表示」より、今回データを取得した期間では多摩都市モノレール線において遅延は発生しなかった。
また「2.weatherに「雨」が含まれている列のみ表示」より、今回の期間中には雨天による遅延が発生しなかった。

###結果

今回の調査では、天候と遅延に関係性は見出すことができなかった。
考察についてはGoogle Slideにて行う。